In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns 
%matplotlib inline

In [2]:
assessments = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-d-nichiporenko/intermediate project/assessments.csv')

In [4]:
assessments.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight
0,AAA,2013J,1752,TMA,19.0,10.0
1,AAA,2013J,1753,TMA,54.0,20.0
2,AAA,2013J,1754,TMA,117.0,20.0
3,AAA,2013J,1755,TMA,166.0,20.0
4,AAA,2013J,1756,TMA,215.0,30.0


In [5]:
courses = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-d-nichiporenko/intermediate project/courses.csv')

In [6]:
courses.head()

,code_module,code_presentation,module_presentation_length
0,AAA,2013J,268
1,AAA,2014J,269
2,BBB,2013J,268
3,BBB,2014J,262
4,BBB,2013B,240


In [7]:
studentAssessment = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-d-nichiporenko/intermediate project/studentAssessment.csv')

In [8]:
studentAssessment.head()

,id_assessment,id_student,date_submitted,is_banked,score
0,1752,11391,18,0,78.0
1,1752,28400,22,0,70.0
2,1752,31604,17,0,72.0
3,1752,32885,26,0,69.0
4,1752,38053,19,0,79.0


In [9]:
studentRegistration = pd.read_csv('/mnt/HC_Volume_18315164/home-jupyter/jupyter-d-nichiporenko/intermediate project/studentRegistration.csv')

In [10]:
studentRegistration.head()

,code_module,code_presentation,id_student,date_registration,date_unregistration
0,AAA,2013J,11391,-159.0,NaN
1,AAA,2013J,28400,-53.0,NaN
2,AAA,2013J,30268,-92.0,12.0
3,AAA,2013J,31604,-52.0,NaN
4,AAA,2013J,32885,-176.0,NaN


In [11]:
# Объединим все таблтцы в одну
df = assessments.merge(courses, how='inner', on=['code_module', 'code_presentation'])\
    .merge(studentRegistration, how='inner', on=['code_module', 'code_presentation'])\
    .merge(studentAssessment, how='inner', on=['id_assessment', 'id_student'])
df.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,module_presentation_length,id_student,date_registration,date_unregistration,date_submitted,is_banked,score
0,AAA,2013J,1752,TMA,19.0,10.0,268,11391,-159.0,NaN,18,0,78.0
1,AAA,2013J,1752,TMA,19.0,10.0,268,28400,-53.0,NaN,22,0,70.0
2,AAA,2013J,1752,TMA,19.0,10.0,268,31604,-52.0,NaN,17,0,72.0
3,AAA,2013J,1752,TMA,19.0,10.0,268,32885,-176.0,NaN,26,0,69.0
4,AAA,2013J,1752,TMA,19.0,10.0,268,38053,-110.0,NaN,19,0,79.0


# Есть два типа курсов: первый заканчивается  экзаменом, второй состоит из различных тестирований, но финального испытания нет.


Определим название предметов, в которых имеются экзамены.

In [12]:
modules_with_exam = list(set(df.query('assessment_type == "Exam"').code_module.to_list()))
modules_with_exam

['DDD', 'CCC']

Определим триместры, в которых есть экзамены.

In [13]:
presentation_with_exam = list(set(df.query('assessment_type == "Exam"').code_presentation.to_list()))
presentation_with_exam

['2014J', '2013B', '2013J', '2014B']

In [14]:
# Кол-во человек на курсе всего
df.groupby(['code_module', 'code_presentation'], as_index=False).id_student.nunique()

,code_module,code_presentation,id_student
0,AAA,2013J,365
1,AAA,2014J,340
2,BBB,2013B,1369
3,BBB,2013J,1710
4,BBB,2014B,1205
5,BBB,2014J,1793
6,CCC,2014B,1415
7,CCC,2014J,1998
8,DDD,2013B,1065
9,DDD,2013J,1507


In [15]:
df = df.drop_duplicates()

# 1. Сколько студентов успешно сдали только один курс? (Успешная сдача — это зачёт по курсу на экзамене).

In [16]:
df.query('is_banked == 0 and assessment_type == "Exam" and score >=40')\
            .groupby('id_student', as_index=False)\
            .agg({'code_module': 'count'})\
            .rename(columns={'code_module': 'count_success_of_module'})\
            .query('count_success_of_module == 1')\
            .count_success_of_module.sum()
# 3802 человека сдали успешно только один курс

3802

# 2. Выяви самый сложный и самый простой экзамен: найди курсы и экзамены в рамках курса, которые обладают самой низкой и самой высокой завершаемостью.

In [17]:
# Создадим таблицу предметов с кол-вом успешных экзаменов
df_success_exams = df.query('assessment_type == "Exam" and score>=40')\
                     .groupby(['code_module', 'code_presentation'], as_index=False)\
                     .score.count()\
                     .rename(columns={'score': 'success_exams'})
df_success_exams

,code_module,code_presentation,success_exams
0,CCC,2014B,664
1,CCC,2014J,1019
2,DDD,2013B,504
3,DDD,2013J,878
4,DDD,2014B,485
5,DDD,2014J,842


In [18]:
# Создадим таблицу с общим кол-вом экзаменов по каждому предмету
df_count_exams = df.query('assessment_type == "Exam"')\
                     .groupby(['code_module', 'code_presentation'], as_index=False)\
                     .score.count()\
                     .rename(columns={'score': 'count_exams'})
df_count_exams

,code_module,code_presentation,count_exams
0,CCC,2014B,747
1,CCC,2014J,1168
2,DDD,2013B,602
3,DDD,2013J,968
4,DDD,2014B,524
5,DDD,2014J,950


In [19]:
# Соединим таблицы 
df_success_count_exams = df_success_exams.merge(df_count_exams, how='inner', \
                                                on=['code_module', 'code_presentation'])

# Создадим столбец с процентом завершаемости курса
df_success_count_exams['percent_completeness'] = round(df_success_count_exams.success_exams / df_success_count_exams.count_exams * 100, 3)
df_success_count_exams

,code_module,code_presentation,success_exams,count_exams,percent_completeness
0,CCC,2014B,664,747,88.889
1,CCC,2014J,1019,1168,87.243
2,DDD,2013B,504,602,83.721
3,DDD,2013J,878,968,90.702
4,DDD,2014B,485,524,92.557
5,DDD,2014J,842,950,88.632


In [20]:
# Самый простой экзамен
df_success_count_exams = df_success_count_exams.sort_values('percent_completeness', ascending=False)
df_success_count_exams.head(1)

,code_module,code_presentation,success_exams,count_exams,percent_completeness
4,DDD,2014B,485,524,92.557


In [21]:
# Самый сложный экзамен
df_success_count_exams = df_success_count_exams.sort_values('percent_completeness')
df_success_count_exams.head(1)

,code_module,code_presentation,success_exams,count_exams,percent_completeness
2,DDD,2013B,504,602,83.721


# 3. По каждому предмету определи средний срок сдачи экзаменов (под сдачей понимаем последнее успешное прохождение экзамена студентом)

In [22]:
# Также оставим столбец module_presentation_length,
# чтобы увидеть разницу с продолжительностью семестра.
df.query("assessment_type == 'Exam' and score>=40")\
    .groupby(['code_module', 'code_presentation', 'module_presentation_length'], as_index=False)\
    .date_submitted.mean()\
    .rename(columns={'date_submitted': 'mean_days_for_pass'})

,code_module,code_presentation,module_presentation_length,mean_days_for_pass
0,CCC,2014B,241,231.581325
1,CCC,2014J,269,244.404318
2,DDD,2013B,240,230.164683
3,DDD,2013J,261,239.509112
4,DDD,2014B,241,234.936082
5,DDD,2014J,262,242.804038


# 4. Выяви самые популярные предметы (ТОП-3) по количеству регистраций на них. А также предметы с самым большим оттоком (ТОП-3)

In [23]:
df.head()

,code_module,code_presentation,id_assessment,assessment_type,date,weight,module_presentation_length,id_student,date_registration,date_unregistration,date_submitted,is_banked,score
0,AAA,2013J,1752,TMA,19.0,10.0,268,11391,-159.0,NaN,18,0,78.0
1,AAA,2013J,1752,TMA,19.0,10.0,268,28400,-53.0,NaN,22,0,70.0
2,AAA,2013J,1752,TMA,19.0,10.0,268,31604,-52.0,NaN,17,0,72.0
3,AAA,2013J,1752,TMA,19.0,10.0,268,32885,-176.0,NaN,26,0,69.0
4,AAA,2013J,1752,TMA,19.0,10.0,268,38053,-110.0,NaN,19,0,79.0


In [24]:
# Отберем только нужные нам столбцы и удалим дупликаты.
df_2 = df[['code_module', 'code_presentation', 'id_student','module_presentation_length','date_registration', 'date_unregistration']] 
df_2 = df_2.drop_duplicates()
df_2.head()

,code_module,code_presentation,id_student,module_presentation_length,date_registration,date_unregistration
0,AAA,2013J,11391,268,-159.0,NaN
1,AAA,2013J,28400,268,-53.0,NaN
2,AAA,2013J,31604,268,-52.0,NaN
3,AAA,2013J,32885,268,-176.0,NaN
4,AAA,2013J,38053,268,-110.0,NaN


In [25]:
# ТОП 3 курса по регистрациям экзаменов (берем все курсы).
# Создаем колонку, где 1 будет, если человек зарегистрировался на курс. Суммируем эти значения. 
df_2['reg'] = df_2.date_registration.notna()
df_reg = df_2.groupby(['code_module', 'code_presentation'], as_index=False)\
    .agg({'reg': 'sum'})\
    .sort_values('reg', ascending=False)
df_reg.head(3)

,code_module,code_presentation,reg
7,CCC,2014J,1998
16,FFF,2013J,1874
18,FFF,2014J,1843


In [26]:
# ТОП 3 курса по оттокам экзаменов (берем все курсы).
# Создаем колонку, где 1 будет, если человек не зарегистрировался на курс. Суммируем эти значения.
df_2['unreg'] = df_2.date_unregistration.notna()#Создаем колонку, где 1 будет, если человек отписался от курса. Суммируем эти значения. 
df_unreg = df_2.groupby(['code_module', 'code_presentation'], as_index=False)\
    .agg({'unreg': 'sum'})\
    .sort_values('unreg', ascending=False)
df_unreg.head(3)

,code_module,code_presentation,unreg
7,CCC,2014J,607
6,CCC,2014B,450
18,FFF,2014J,373


In [27]:
# ТОП 3 курса по регистрациям с экзаменами в конце

# Определим триместры, в которых есть экзамен
module_with_exams = df.query('assessment_type == "Exam"').code_module.unique()
# Определим семестры, в которых есть экзамены
presentation_with_exams = df.query('assessment_type == "Exam"').code_presentation.unique()

df_only_exams_reg = df_reg.query('code_module in @module_with_exams and  code_presentation in @presentation_with_exams')
df_only_exams_reg.head(3)

,code_module,code_presentation,reg
7,CCC,2014J,1998
9,DDD,2013J,1506
11,DDD,2014J,1427


In [28]:
# ТОП 3 курса по оттоку с экзаменами в конце
df_only_exams_outflow = df_unreg.query('code_module in @module_with_exams and  code_presentation in @presentation_with_exams')
df_only_exams_outflow.head(3)

,code_module,code_presentation,unreg
7,CCC,2014J,607
6,CCC,2014B,450
9,DDD,2013J,327


# 5. Используя pandas, в период с начала 2013 по конец 2014 выяви семестр с самой низкой завершаемостью курсов и самыми долгими средними сроками сдачи курсов.

In [29]:
# Посчитаем успешные экзамены по семестрам
df_success_count_exams = df_success_exams.merge(df_count_exams, how='inner', \
                                                on=['code_module', 'code_presentation'])
df_presentation_success = df_success_count_exams.groupby('code_presentation', as_index=False)\
    .success_exams.sum()
df_presentation_success

,code_presentation,success_exams
0,2013B,504
1,2013J,878
2,2014B,1149
3,2014J,1861


In [30]:
# Посчитаем кол-во экзаменов по семестрам
df_presentation_count_exams = df_success_count_exams.groupby('code_presentation', as_index=False)\
    .count_exams.sum()
df_presentation_count_exams

,code_presentation,count_exams
0,2013B,602
1,2013J,968
2,2014B,1271
3,2014J,2118


In [31]:
# Объедими две таблицы
# Создадим столбец с процентом завершенности по курсу 
df_percentege_of_success = df_presentation_success.merge(df_presentation_count_exams,\
                                                        how='inner',\
                                                        on='code_presentation')
df_percentege_of_success['per_success'] = df_percentege_of_success.success_exams / df_percentege_of_success.count_exams * 100
# Сортируем процент по возрастанию и выводим первую строчку. В этой строчке содержится семестр с самой низкой завершаемостью.
df_percentege_of_success.sort_values('per_success').head(1)

,code_presentation,success_exams,count_exams,per_success
0,2013B,504,602,83.72093


In [32]:
# Семестр с самым долгим средним сдачи курсов
df.query('assessment_type =="Exam"')\
    .groupby('code_presentation', as_index=False)\
    .date_submitted.mean()\
    .sort_values('date_submitted', ascending=False)\
    .head(1)

,code_presentation,date_submitted
3,2014J,243.674221


# 6.  Часто для качественного анализа аудитории использую подходы, основанные на сегментации. Используя python, построй адаптированные RFM-кластеры студентов, чтобы качественно оценить свою аудиторию. В адаптированной кластеризации можешь выбрать следующие метрики: R - среднее время сдачи одного экзамена, F - завершаемость курсов, M - среднее количество баллов, получаемое за экзамен. Подробно опиши, как ты создавал кластеры. Для каждого RFM-сегмента построй границы метрик recency, frequency и monetary для интерпретации этих кластеров.

In [33]:
# Функции, которая присваивает ранк от 1 до 5 каждой строчке
# Эта функция отвечает за средний балл студента
# Чем больше значение, тем значение ранка ближе к 1
def m_score(x, c):
    if x <= quintiles[c][.2]:
        return 5
    elif x <= quintiles[c][.4]:
        return 4
    elif x <= quintiles[c][.6]:
        return 3
    elif x <= quintiles[c][.8]:
        return 2
    else:
        return 1

# Эта фунция отвечает за среднее время сдачи студента.
# Здесь чем лучше значение, тем ближе оно к 1.
def r_score(x):
    if x <= quintiles['date_submitted'][.2]:
        return 1
    elif x <= quintiles['date_submitted'][.4]:
        return 2
    elif x <= quintiles['date_submitted'][.6]:
        return 3
    elif x <= quintiles['date_submitted'][.8]:
        return 4
    else:
        return 5  

# Эта фунция отвечает за заверенность студентом курсов. 
# Чем ближе к 1, тем лучше
def f_score(x):
    if x == 100:
        return 1 
    elif 50 <= x < 100:
        return 2
    else:
        return 3

In [34]:
# Создадим дф, где указано среднее время сдачи экзамена студентом
df_rfm_r = df.query('assessment_type == "Exam"')\
    .groupby(['id_student'], as_index=False)\
    .date_submitted.mean()
df_rfm_r

,id_student,date_submitted
0,23698,243.0
1,24213,236.0
2,27116,243.0
3,28046,237.0
4,28787,243.0
...,...,...
4628,2694886,236.0
4629,2694933,230.0
4630,2695608,237.0
4631,2697181,230.0


In [35]:
df.query('assessment_type == "Exam" and score <40')

,code_module,code_presentation,id_assessment,assessment_type,date,weight,module_presentation_length,id_student,date_registration,date_unregistration,date_submitted,is_banked,score
52941,CCC,2014B,24290,Exam,NaN,100.0,241,118098,-26.0,NaN,234,0,24.0
52971,CCC,2014B,24290,Exam,NaN,100.0,241,206599,-24.0,NaN,230,0,38.0
52973,CCC,2014B,24290,Exam,NaN,100.0,241,227752,-41.0,NaN,234,0,18.0
52974,CCC,2014B,24290,Exam,NaN,100.0,241,230761,-12.0,NaN,234,0,30.0
52985,CCC,2014B,24290,Exam,NaN,100.0,241,261265,-43.0,NaN,230,0,38.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95931,DDD,2014J,25368,Exam,NaN,100.0,262,2232340,-112.0,NaN,249,0,36.0
95947,DDD,2014J,25368,Exam,NaN,100.0,262,2391689,-101.0,NaN,243,0,38.0
95951,DDD,2014J,25368,Exam,NaN,100.0,262,2459418,-135.0,NaN,243,0,27.0
95965,DDD,2014J,25368,Exam,NaN,100.0,262,2561811,-95.0,NaN,242,0,33.0


In [36]:
# Создадим df с кол-вом успешных экзаменов студента
df_success_exams_student = df.query('assessment_type == "Exam" and score>=40')\
                     .groupby(['id_student'], as_index=False)\
                     .score.count()\
                     .rename(columns={'score': 'success_exams'})
# Создадим df с кол-вом экзаменов по студенту
df_count_exams_student = df.query('assessment_type == "Exam"')\
                     .groupby(['id_student'], as_index=False)\
                     .score.count()\
                     .rename(columns={'score': 'count_exams'})

# Объединим таблицы
df_rfm_f = df_success_exams_student.merge(df_count_exams_student,\
                                         how='right',\
                                         on='id_student')
# 
# Создадим столбец с процентом завершенности
df_rfm_f['per_compl'] = df_rfm_f.success_exams / df_rfm_f.count_exams * 100
df_rfm_f['success_exams'] = df_rfm_f.success_exams.fillna(0)
df_rfm_f['per_compl'] = df_rfm_f.per_compl.fillna(0)

df_rfm_f.head()

,id_student,success_exams,count_exams,per_compl
0,23698,1.0,1,100.0
1,24213,1.0,1,100.0
2,27116,1.0,1,100.0
3,28046,1.0,1,100.0
4,28787,1.0,1,100.0


In [37]:
# Создадим дф, где указано средний балл за экзамен студента
df_rfm_m = df.query('assessment_type == "Exam"')\
    .groupby(['id_student'], as_index=False)\
    .score.mean()
df_rfm_m

,id_student,score
0,23698,80.0
1,24213,58.0
2,27116,96.0
3,28046,40.0
4,28787,44.0
...,...,...
4628,2694886,69.0
4629,2694933,73.0
4630,2695608,73.0
4631,2697181,80.0


In [38]:
# Объединим все таблицы 
df_rfm = df_rfm_r.merge(df_rfm_m, how='inner', on=['id_student'])\
    .merge(df_rfm_f, how='inner', on=['id_student'])\
    .rename(columns={'score': 'avg_score_student'})
df_rfm

,id_student,date_submitted,avg_score_student,success_exams,count_exams,per_compl
0,23698,243.0,80.0,1.0,1,100.0
1,24213,236.0,58.0,1.0,1,100.0
2,27116,243.0,96.0,1.0,1,100.0
3,28046,237.0,40.0,1.0,1,100.0
4,28787,243.0,44.0,1.0,1,100.0
...,...,...,...,...,...,...
4628,2694886,236.0,69.0,1.0,1,100.0
4629,2694933,230.0,73.0,1.0,1,100.0
4630,2695608,237.0,73.0,1.0,1,100.0
4631,2697181,230.0,80.0,1.0,1,100.0


In [39]:
# Найдем квантили для параметров кроме завершенности курса. Для завершенности приеним функцию, которая описана выше
quintiles = df_rfm[['date_submitted', 'avg_score_student']]\
    .quantile([.2, .4, .6, .8]).to_dict()
quintiles

{'date_submitted': {0.2: 231.0, 0.4: 237.0, 0.6: 242.0, 0.8: 243.5},
 'avg_score_student': {0.2: 46.200000000000045,
  0.4: 60.0,
  0.6: 72.0,
  0.8: 84.0}}

In [40]:
# Добавляем в датафрейм столбцы с R, F, M оценками для каждого экзамена.
# Показатель R оценки меньше, если продолжительность меньше
# Чем больше F оценка, тем хуже процент заершенности
# Чем больше M оценка, тем хуже средний балл
df_rfm['R'] = df_rfm['date_submitted'].apply(lambda x: r_score(x))
df_rfm['F'] = df_rfm['per_compl'].apply(lambda x: f_score(x))
df_rfm['M'] = df_rfm['avg_score_student'].apply(lambda x: m_score(x, 'avg_score_student'))
df_rfm['RFM Score'] = df_rfm['R'].map(str) + df_rfm['F'].map(str) + df_rfm['M'].map(str)
df_rfm

,id_student,date_submitted,avg_score_student,success_exams,count_exams,per_compl,R,F,M,RFM Score
0,23698,243.0,80.0,1.0,1,100.0,4,1,2,412
1,24213,236.0,58.0,1.0,1,100.0,2,1,4,214
2,27116,243.0,96.0,1.0,1,100.0,4,1,1,411
3,28046,237.0,40.0,1.0,1,100.0,2,1,5,215
4,28787,243.0,44.0,1.0,1,100.0,4,1,5,415
...,...,...,...,...,...,...,...,...,...,...
4628,2694886,236.0,69.0,1.0,1,100.0,2,1,3,213
4629,2694933,230.0,73.0,1.0,1,100.0,1,1,2,112
4630,2695608,237.0,73.0,1.0,1,100.0,2,1,2,212
4631,2697181,230.0,80.0,1.0,1,100.0,1,1,2,112


In [41]:
# Опредлеим сегменты для каждого RFM score
segt_map = {
    r'[1-2]1': 'Хороший студент',
    r'[3-4]1': 'Хороший студент',
    r'[5]1': 'Хороший студент, но нужно больше времени на раскачку',
    r'[1-2]2': 'Средний студент',
    r'[3-4]2': 'Средний студент',
    r'[5]2': 'Средний студент, котрому требуется больше времени',
    r'[1-2]3': 'Студент не смог сдать ни один экзамен',
    r'[3-4]3': 'Студент не смог сдать ни один экзамен',
    r'[5]3': 'Студент не смог сдать ни один экзамен'
}
df_rfm['Segment'] = df_rfm['R'].map(str) + df_rfm['F'].map(str)
df_rfm['Segment'] = df_rfm['Segment'].replace(segt_map, regex=True)
df_rfm.head()

,id_student,date_submitted,avg_score_student,success_exams,count_exams,per_compl,R,F,M,RFM Score,Segment
0,23698,243.0,80.0,1.0,1,100.0,4,1,2,412,Хороший студент
1,24213,236.0,58.0,1.0,1,100.0,2,1,4,214,Хороший студент
2,27116,243.0,96.0,1.0,1,100.0,4,1,1,411,Хороший студент
3,28046,237.0,40.0,1.0,1,100.0,2,1,5,215,Хороший студент
4,28787,243.0,44.0,1.0,1,100.0,4,1,5,415,Хороший студент


# Описания сегментов
Хороший студент: сдал все экзамены + хорошее среднее время

Хороший студент, но нужно больше времени на раскачку: сдал все экзамены + среднее время больше, чем у просто хорошего студента

Средний студент: сдал половину экзаменов + хорошее среднее время

Средний студент, котрому требуется больше времени: сдал половину экзаменов + среднее время больше, чем у просто среднего студента

Студент не смог сдать ни один экзамен: не сдал ни одного предмета, время рассматривать нет смысла

Данная сегментация имеет минусы. Например, среднее время сдачи курса зависит от предмета.

In [42]:
df_rfm.groupby('Segment', as_index=False).id_student.count()

,Segment,id_student
0,Средний студент,28
1,Студент не смог сдать ни один экзамен,536
2,Хороший студент,3270
3,"Хороший студент, но нужно больше времени на ра...",799


Тестовая запись для коммита!